In [3]:
import os

import numpy as np
import jieba
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Sequential
from keras.callbacks import LambdaCallback, ModelCheckpoint

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
weight_path = 'data/model/Word2Vec_v1.4/w2v.model.bin'
model = Word2Vec.load(weight_path)

# text data preprocessing

In [5]:
def preprocess_text_data(raw_docs):
    # jieba settings
    jieba.enable_parallel(6)
    jieba.set_dictionary("data/jieba_dict/dict.txt.big")
    jieba.load_userdict("data/jieba_dict/中央機構.dict")
    jieba.load_userdict("data/jieba_dict/名人錄.dict")
    jieba.load_userdict("data/jieba_dict/專有名詞.dict")
    jieba.load_userdict("data/jieba_dict/縣市區鄉鎮.dict")
    # 用來存放分詞後的結果
    preprocessed_documents = []
    for index, raw_doc in enumerate(raw_docs, 0):
        if index % 2000 == 0:
            print("current document index:{}".format(index))
        doc = " ".join(jieba.cut(raw_doc))
        preprocessed_documents.append(doc)
    return preprocessed_documents

In [6]:
# 載入 corpus
corpus_path = "data/text/big_data/corpus"
file_name = os.listdir(corpus_path)[3]

with open(corpus_path + "/" + file_name, "r", encoding="utf-8") as content:
    document_list = [line.strip().replace(' ', '') for line in content]

preprocessed_documents = preprocess_text_data(document_list)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.542 seconds.
Prefix dict has been built succesfully.
Building prefix dict from /root/jupyter-project/text_generation/data/jieba_dict/dict.txt.big ...
Loading model from cache /tmp/jieba.ubbac126d8c196e35a5c759d30d825abf.cache
Loading model cost 1.040 seconds.
Prefix dict has been built succesfully.


current document index:0
current document index:2000
current document index:4000
current document index:6000
current document index:8000
current document index:10000
current document index:12000
current document index:14000
current document index:16000
current document index:18000
current document index:20000
current document index:22000
current document index:24000
current document index:26000
current document index:28000
current document index:30000
current document index:32000
current document index:34000
current document index:36000
current document index:38000
current document index:40000
current document index:42000
current document index:44000
current document index:46000
current document index:48000
current document index:50000


Process ForkPoolWorker-2:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-1:
Process ForkPoolWorker-6:
Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessin

In [7]:
t = Tokenizer()
t.fit_on_texts(preprocessed_documents)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(preprocessed_documents)
# pad documents to a max length
max_length = 500
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[   608 111059  18937 ...     81    114    193]
 [    17  16433    196 ...  12761     16   6962]
 [   129     17    180 ...      0      0      0]
 ...
 [   490     60  38364 ...      0      0      0]
 [  2417      2   1721 ...    201     58    485]
 [   159   4062   3327 ...      0      0      0]]


In [8]:
index2word = dict((v, k) for k, v in t.word_index.items())
len(index2word)

230466

In [9]:
# created pretrained embeding matrix
embedding_matrix = np.zeros((vocab_size, 300))

for word, index in t.word_index.items():
    try:
        vector = model.wv.get_vector(word)
        embedding_matrix[index] = vector
    except Exception as KeyError:
        print(f"{word} not found!")

至於 not found!
　 not found!
黨產會 not found!
兩岸關 not found!
ndgkeywordh4 not found!
隨組織 not found!
阿布鐸 not found!
2014 not found!
澳電廠 not found!
徐國勇 not found!
臉書上 not found!
競選團隊 not found!
ndgkeyword not found!
ndgkeywordh3 not found!
ndgkeywordh3a not found!
競選辦 not found!
並沒有 not found!
美國洛杉磯 not found!
選總統 not found!
國民黨團 not found!
日舉行 not found!
李憲章 not found!
kolasyotaka not found!
一開始 not found!
將持續 not found!
台灣民眾 not found!
職典禮 not found!
新聞授 not found!
權來源 not found!
美國國 not found!
考紀會 not found!
國民黨籍 not found!
東森新聞 not found!
國防授 not found!
文傳會 not found!
連勝文 not found!
將台灣 not found!
日召開 not found!
社記者 not found!
對民進 not found!
全台灣 not found!
會不會 not found!
婦銅 not found!
青運主 not found!
eaoc not found!
負責任 not found!
遴委會 not found!
許多人 not found!
當選人 not found!
臉書粉 not found!
酈英傑 not found!
讓民眾 not found!
姚文智今 not found!
議員候 not found!
張善政 not found!
當選市 not found!
機農業 not found!
伴關 not found!
這幾年 not found!
理委員會 not found!
報導民 not found!
姚文智辦 not found!
遴選委員會 not found!
姚文智

邀請民眾 not found!
預算會期 not found!
中國則 not found!
法規定 not found!
同選區 not found!
財服務處 not found!
三軍統帥 not found!
勞保年 not found!
最後都 not found!
有共識 not found!
發生過 not found!
捷運站 not found!
大選前 not found!
開發案 not found!
挪台灣 not found!
員林國宅 not found!
伴侶關 not found!
起義的 not found!
黨產會會 not found!
日登記 not found!
corygardner not found!
卞金峰 not found!
這番言論 not found!
經貿關 not found!
伊茲米爾 not found!
市合 not found!
維安人員 not found!
這兩位 not found!
經國號 not found!
參選以 not found!
kimjongun not found!
相關訊息 not found!
籍候 not found!
區農會 not found!
最後關頭 not found!
當我們 not found!
發言人何 not found!
區里長 not found!
部舉 not found!
留子孫 not found!
改釋 not found!
課徵糖稅 not found!
統姆南 not found!
創產業 not found!
蔡總統致 not found!
參加過 not found!
鄉民大會 not found!
前議長 not found!
中國來 not found!
izmir not found!
仍無法 not found!
國民黨則 not found!
樓護理 not found!
政黨法 not found!
nelsonchamisa not found!
robertlighthizer not found!
下一個 not found!
東協國家 not found!
捍衛國 not found!
競選主軸 not found!
選民對 not found!
華長 not found!
化幼 not found!
並結合 

非事實 not found!
姚文智拉 not found!
遭民眾 not found!
運綠線 not found!
土典禮 not found!
已經將 not found!
大遊行 not found!
拚選戰 not found!
更務實 not found!
並補充 not found!
無所適 not found!
指國民黨 not found!
反輔選 not found!
之議題 not found!
將永遠 not found!
審判決 not found!
姚文智自 not found!
同時間 not found!
進行國 not found!
人員有 not found!
輔選列 not found!
連一個 not found!
但大陸 not found!
投地區 not found!
創新產業 not found!
以美國 not found!
這幾位 not found!
唐聲揚 not found!
緊鑼密 not found!
人員到 not found!
疑柯 not found!
調初 not found!
談判時 not found!
軍艦隊 not found!
凱耶塔諾 not found!
之黨籍 not found!
鄉民們 not found!
中國統 not found!
哲選情 not found!
關於島嶼 not found!
講這個 not found!
連議員 not found!
會大樓 not found!
美國要 not found!
日選舉 not found!
國軍在 not found!
以後再 not found!
余莓莓 not found!
機會時 not found!
勞資會議 not found!
綠建築 not found!
和員工 not found!
詞歸責 not found!
體育處 not found!
晶圓出 not found!
項惠台 not found!
點關鍵 not found!
美期間 not found!
無協議 not found!
選戰主 not found!
大醫院 not found!
新貿易 not found!
將超過 not found!
開車載 not found!
貿動態 not found!
業國際 not found!
小樹屋 n

高思博喊 not found!
改團體 not found!
經後備 not found!
起義務役 not found!
續動員 not found!
召訓練 not found!
檢討精 not found!
進後備 not found!
海戰役 not found!
海軍樂 not found!
最精準 not found!
姚文智並 not found!
輛汽車 not found!
讀大學 not found!
人憂心 not found!
  not found!
視察福 not found!
外運動 not found!
捍衛國民黨 not found!
總督導 not found!
回憶過 not found!
總處理 not found!
國家階層 not found!
相關言論 not found!
最寶貴 not found!
億賤 not found!
雜嚴峻 not found!
結勝選 not found!
違法變 not found!
僱員工 not found!
愈況 not found!
舉行告 not found!
青運到 not found!
萬元罰 not found!
賓海 not found!
原規劃 not found!
激動處 not found!
a型 not found!
哲簽 not found!
抵達會場 not found!
李耀華 not found!
萬一國民黨 not found!
親自為 not found!
比蘇院 not found!
太感動 not found!
並準備 not found!
高達近 not found!
典禮延 not found!
參與會議 not found!
兩黃金及 not found!
哪有後來 not found!
變本加 not found!
創新升級 not found!
咱台灣 not found!
將我國 not found!
穩腳步 not found!
並不只 not found!
經晉 not found!
江系統 not found!
黨議會 not found!
產會委員 not found!
辦檢討會 not found!
愛心會 not found!
姚民調 not found!
權力關 not found!
台灣鳳 not found!
門未

行見證 not found!
辜民眾 not found!
議持續 not found!
還要繳 not found!
違憲疑慮 not found!
藍營市 not found!
螺絲像過 not found!
輪選舉 not found!
方極力 not found!
具戰略 not found!
報導過 not found!
組電池 not found!
raiba not found!
aiot not found!
thediplomat not found!
個區域 not found!
千萬點閱 not found!
家承貸 not found!
醫療業 not found!
並聆 not found!
行動來 not found!
大翻轉 not found!
前陸委會 not found!
國人共識 not found!
累計實 not found!
壓作 not found!
蔡政績 not found!
並強烈 not found!
並落實 not found!
決現 not found!
萬元調 not found!
並規定 not found!
線糊 not found!
英九並 not found!
當醫生 not found!
宣導會 not found!
偉哲萬分 not found!
或民間 not found!
執政是 not found!
踐踏台灣 not found!
美國長 not found!
如美國 not found!
許立民溝 not found!
並不滿意 not found!
務會議 not found!
建設類 not found!
總計畫 not found!
cnpc not found!
偉哲公開 not found!
姚文智表達 not found!
區觀光 not found!
還創造 not found!
經濟弱 not found!
事務委員 not found!
選民將 not found!
柯競 not found!
英九特別 not found!
世大運選 not found!
工資委員會 not found!
經表態 not found!
並高舉 not found!
但議員 not found!
藍議員 not found!
單位給 not found!
制倉 not found!
遴選

...

以足夠 not found!
以常規 not found!
行動將 not found!
程攻擊 not found!
可破壞 not found!
作戰節 not found!
防衛有 not found!
可獨立 not found!
反擊報 not found!
數國家作 not found!
業務統 not found!
亞太鄰 not found!
普傾向 not found!
趁美中 not found!
而國際 not found!
飆破國 not found!
接風險 not found!
接信貸 not found!
貸業務 not found!
著火時 not found!
台灣入 not found!
遺產名 not found!
觀光展 not found!
鄰國關 not found!
臨岸邊 not found!
濃濃的魚 not found!
強迫環 not found!
鐵桃園 not found!
先恭維市 not found!
經允諾 not found!
有動工 not found!
這兩站 not found!
中壢線 not found!
化路線 not found!
閔未 not found!
職下 not found!
大薪 not found!
資報 not found!
酬會議 not found!
議遲 not found!
行國立 not found!
大學產學 not found!
文大學 not found!
生四人 not found!
還沒繳 not found!
實納稅 not found!
國民們 not found!
背後結 not found!
收三萬 not found!
文大解 not found!
以大漢溪 not found!
廣大及 not found!
橋夜 not found!
關委員會 not found!
院質 not found!
官員級 not found!
更高級 not found!
躁動緊 not found!
親變成 not found!
惡化罷 not found!
報總主筆 not found!
之經濟 not found!
農之經營 not found!
王俊凱親 not found!
近瓊 not found!
勢崎嶇 not found!
mikefarnw

員時常 not found!
詢協助 not found!
身說法 not found!
軍之路 not found!
1691294560993473 not found!
務學生 not found!
認識國 not found!
蔡中泰 not found!
修護科 not found!
水電技術科 not found!
國軍所 not found!
曾佩鈺 not found!
營隊裡 not found!
史偉勤 not found!
中美過 not found!
貿爭 not found!
此後中國 not found!
防範系 not found!
點推動 not found!
檢視陸 not found!
訓後回 not found!
應在國 not found!
火協機 not found!
制運作 not found!
納入後續 not found!
應未來 not found!
防大學陸 not found!
參謀學院 not found!
地馬拉籍 not found!
邰柏亞 not found!
邰員術 not found!
邰員表達 not found!
協助後續 not found!
返國後發揮 not found!
部轄屬 not found!
握彈 not found!
保險銷 not found!
按訓練 not found!
王善儒 not found!
姜王瀚及 not found!
李軍彥則 not found!
作練習 not found!
生訓練 not found!
日大動作 not found!
來黃益 not found!
讓黃益 not found!
選人敢 not found!
軍東引 not found!
海軍雷達 not found!
及大隊 not found!
循環初 not found!
賽後由海 not found!
官夏 not found!
頒冠 not found!
軍錦旗 not found!
東引海 not found!
聯和大陸 not found!
學術圈 not found!
辦僅 not found!
研所長 not found!
張冠華代 not found!
記黃曉薇 not found!
平關注 not found!
互動求 not found!
穩不求 not found

競價得 not found!
標費率 not found!
鞋作 not found!
雙原價 not found!
雙半價 not found!
swireblueocean not found!
隊據 not found!
國產風 not found!
電運維船 not found!
crewtransfervessel not found!
風電港 not found!
wteam not found!
廠商能 not found!
風機安裝 not found!
電海纜 not found!
marineteam not found!
塔桶 not found!
減資元 not found!
5gw not found!
已民營化 not found!
將風機 not found!
碼頭載 not found!
將運維船 not found!
拖運到 not found!
機塔柱 not found!
外船 not found!
風電設 not found!
掌離岸 not found!
缺相關 not found!
中國驗 not found!
船協會 not found!
輔導國際 not found!
安裝動態 not found!
dynamicposition not found!
日後萬 not found!
哲失 not found!
洪耀福否 not found!
認民 not found!
運國手 not found!
身議 not found!
小組要 not found!
哲邊 not found!
並表決 not found!
出議場 not found!
包輕快 not found!
小平先 not found!
大會備 not found!
快樂放學 not found!
暴紅學姐 not found!
近來民調 not found!
過去礙 not found!
生買下 not found!
地護航 not found!
吉仲當然 not found!
音寧帶 not found!
著勝利 not found!
日後蔡 not found!
日柯粉 not found!
選民不分 not found!
手陳學 not found!
賴神來 not found!
談不備詢 not found!
議員拿來 not found!
罰吳

因小時候 not found!
一棟屬 not found!
飲料業 not found!
學旁 not found!
氣連開 not found!
吳政學過 not found!
打游擊戰 not found!
許多貴 not found!
吳政學則 not found!
真誠態度 not found!
七顧 not found!
先後拜 not found!
亞太會 not found!
鄭師傅 not found!
而揚言 not found!
網承認 not found!
吳政學將 not found!
具噴槍 not found!
霧式 not found!
區興隆 not found!
池玉蘭親 not found!
許榮民先 not found!
能動工 not found!
因財務 not found!
處工務 not found!
邱盈達 not found!
已委外 not found!
產業館 not found!
市場性 not found!
背後影響 not found!
隨風雨 not found!
對島民 not found!
並無屬 not found!
史章 not found!
獲疾 not found!
挾高人氣 not found!
韓卡司 not found!
如馬祖 not found!
相印機 not found!
書人帶 not found!
節時將 not found!
持活動 not found!
至陳姓 not found!
動聘 not found!
新聞流 not found!
緬甸吝 not found!
展開鎮 not found!
rohingya not found!
逃離家園 not found!
臉書還 not found!
markzuckerberg not found!
projecthoneybadger not found!
葉在當 not found!
參加民間 not found!
師時候 not found!
國賓夕 not found!
大學進 not found!
修碩士 not found!
紐約州律師 not found!
擔任北 not found!
始擔 not found!
間司 not found!
葉慶元剛 not found!
葉早 not found!
委員質

朝鮮動員 not found!
客數量 not found!
訪日遊 not found!
災害給 not found!
發生強 not found!
關西國際 not found!
呂溫泉 not found!
但訪 not found!
大陸團 not found!
中國供 not found!
維持戰 not found!
確保訪 not found!
將首頁 not found!
挺公園 not found!
變造勢 not found!
哲擺 not found!
擺公園 not found!
變選民 not found!
批對岸 not found!
夾中間 not found!
評染紅靠 not found!
品寧帶 not found!
公會將 not found!
歡迎會員 not found!
山公園 not found!
圓夢築家 not found!
蔡鈞安 not found!
踴躍購票 not found!
贈由腦 not found!
麻協會親 not found!
彈頻 not found!
繁射擊 not found!
奮金門 not found!
好漢中 not found!
任團長 not found!
補金門 not found!
幹部擇 not found!
優選任 not found!
楊辛旺 not found!
139t not found!
國軍不分 not found!
老裝備 not found!
龍兆城 not found!
502t not found!
307t not found!
此鴻運 not found!
祝陸 not found!
貝雙方 not found!
徵誠摯 not found!
該勳 not found!
章象 not found!
空拍機 not found!
李正紀還 not found!
懸吊帶 not found!
totalbodyresistanceexercise not found!
雪前 not found!
時太陽能 not found!
雖氣溫 not found!
鏟雪前 not found!
先拿來 not found!
先用電 not found!
水不夠 not found!
水馬桶 not found!
公務員常 not found!
若嘉義氣象 not

In [35]:
# rnn model
rnn_model = Sequential()
rnn_model.add(Embedding(
    vocab_size, 
    300, 
    weights=[embedding_matrix], 
    input_length=max_length-1, # -1 for label y 
    trainable=False
))
rnn_model.add(LSTM(300, return_sequences=True))
rnn_model.add(LSTM(200, dropout=0.5))
rnn_model.add(Dense(units=vocab_size, activation="softmax"))
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 499, 300)          69140100  
_________________________________________________________________
lstm_5 (LSTM)                (None, 499, 300)          721200    
_________________________________________________________________
lstm_6 (LSTM)                (None, 200)               400800    
_________________________________________________________________
dense_4 (Dense)              (None, 230467)            46323867  
Total params: 116,585,967
Trainable params: 47,445,867
Non-trainable params: 69,140,100
_________________________________________________________________


In [11]:
train_x, train_y = padded_docs[:,:-1], padded_docs[:,-1]
print(f"train x shape: {train_x.shape}, train y shape: {train_y.shape}")

train x shape: (51763, 499), train y shape: (51763,)


In [30]:
def sample(preds, temperature=1.0):
    """
    temperature 表示控制 sample 字的多樣性，越高越隨機
    越低則越強化原本預測機率的差距，ex: [0.2, 0.5, 0.3] -> [0.009, 0.91, 0.07]
    """
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_next(text, num_generated=10, temperature=1.0):
    """
    text = '馬英九 今天 吃'
    """
    # only one doc
    encoded_doc = t.texts_to_sequences([text])[0]
    for i in range(num_generated):
        padded_docs = pad_sequences([encoded_doc], maxlen=500-1, padding='post')
        prediction = rnn_model.predict(x=padded_docs)
        index = sample(prediction[0], temperature)
        encoded_doc.append(index)
    return ''.join(index2word.get(index, '') for index in encoded_doc)

def on_epoch_end(epoch, _):
    print('\nGenerating text after epoch: %d' % epoch)
    texts = ['馬英九']
    for text in texts:
        print('%s... -> %s' % (text, generate_next(text, 10, 0.5)))

In [31]:
# define the checkpoint
filepath="pretrained-weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [36]:
import time

start_time = time.perf_counter()
rnn_model.fit(
    train_x, 
    train_y, 
    batch_size=128, 
    epochs=50, 
    callbacks=[LambdaCallback(on_epoch_end=on_epoch_end), checkpoint],
    validation_split=0.2
)
end_time = time.perf_counter()
print(f"total model training time:{end_time-start_time} secs")

Train on 41410 samples, validate on 10353 samples
Epoch 1/50
 1792/41410 [>.............................] - ETA: 18:18 - loss: 11.8623

KeyboardInterrupt: 

In [14]:
rnn_model.load_weights(filepath)
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [29]:
generate_next(['台灣'], num_generated=500, temperature=0.4)

(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 230467)
0
(1, 23

'台灣'